In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import kurtosis, mode
import pandas_profiling
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import _plotly_future_
from _plotly_future_ import v4_subplots

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import seaborn as sns

import matplotlib.pyplot as plt
from datetime import datetime
import random
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

import yaml
#import boto3
from io import BytesIO
import datetime
from datetime import timedelta, datetime, date

hour = timedelta(hours=1)
day = timedelta(days=1)
%matplotlib inline


from itertools import cycle
pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [2]:
path_sell_price= '/Users/asadmolayari/Desktop/my_courses/Kaggle/M5/m5-forecasting-accuracy/sell_prices.csv'
path_trin_val= '/Users/asadmolayari/Desktop/my_courses/Kaggle/M5/m5-forecasting-accuracy/sales_train_validation.csv'
path_cld= '/Users/asadmolayari/Desktop/my_courses/Kaggle/M5/m5-forecasting-accuracy/calendar.csv'
path_sbmssion='/Users/asadmolayari/Desktop/my_courses/Kaggle/M5/m5-forecasting-accuracy/sample_submission.csv'
path_train_eval='/Users/asadmolayari/Desktop/my_courses/Kaggle/M5/m5-forecasting-accuracy/sales_train_evaluation.csv'

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
df_train_2=pd.read_csv(path_train_eval)
df_train_2=reduce_mem_usage(df_train_2)

Mem. usage decreased to 96.13 Mb (78.8% reduction)


In [33]:
df_train=pd.read_csv(path_trin_val)
df_train=reduce_mem_usage(df_train)

Mem. usage decreased to 95.00 Mb (78.7% reduction)


In [6]:
df_price=pd.read_csv(path_sell_price)
df_price=reduce_mem_usage(df_price)

Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [7]:
df_cld=pd.read_csv(path_cld)
df_cld=reduce_mem_usage(df_cld)

Mem. usage decreased to  0.12 Mb (41.9% reduction)


In [8]:
df_sbmission=pd.read_csv(path_sbmssion)
df_sbmission=reduce_mem_usage(df_sbmission)

Mem. usage decreased to  2.09 Mb (84.5% reduction)


# MODELING

## lgbm Model

In [9]:
import dask.dataframe as dd

import lightgbm as lgb
import xgboost as xgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
import gc
import os

In [10]:
def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id',\
                            'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0]\
                                                                            , sales_train_validation.shape[1]))
    #sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    # get only a sample for fst training
    #data = data.loc[nrows:]
    
    # drop some calendar features
    #calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    #breakpoint()
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    #breakpoint()
    gc.collect()
    
    return data


In [11]:
data = melt_and_merge(df_cld, df_price, df_train_2, df_sbmission, nrows = 27500000, merge = True)
gc.collect()
data=reduce_mem_usage(data)

Melted sales train validation has 59181090 rows and 8 columns
> <ipython-input-10-ceccaa1bb25a>(50)melt_and_merge()
-> if merge:
(Pdb) c
Our final dataset to train has 60034810 rows and 22 columns
> <ipython-input-10-ceccaa1bb25a>(61)melt_and_merge()
-> gc.collect()
(Pdb) c
Mem. usage decreased to 7156.71 Mb (0.0% reduction)


In [19]:
del df_cld, df_price, df_train,df_train_2 #df_sbmission

In [ ]:
data.info()

In [ ]:
data.iloc[:1000].groupby(['id'])['sell_price'].mean()

In [ ]:
temp=data[['id','sell_price']]
temp['sell_price']=temp['sell_price'].astype(np.float32)
temp.groupby(['id'])['sell_price'].transform(lambda x: x.pct_change(periods=7)).replace(np.inf,np.nan).fillna(method='ffill')#.dropna() lambda x: x.shift(1)

In [ ]:
data.groupby(['id'])['sell_price'].transform(lambda x: x.pct_change(periods=7))#.replace(np.inf,np.nan).fillna(method='ffill')#.dropna() lambda x: x.shift(1)

In [ ]:
t1=datetime.now()
temp1=data.groupby(['id'])['demand'].apply(lambda x: x.rolling(7).mean())#.replace(np.inf,np.nan).fillna(method='ffill').dropna()
t2=datetime.now()
print(t2-t1)

In [ ]:
t1=datetime.now()
temp2=data.groupby(['id'])['demand'].transform(lambda x: x.rolling(7).mean())#.replace(np.inf,np.nan).fillna(method='ffill').dropna()
t2=datetime.now()
print(t2-t1)

In [ ]:
temp1[temp1!=0].dropna()

In [ ]:
temp2[temp2!=0].dropna()

In [31]:
data

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,id_encode
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,NaN,28740
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,NaN,28760
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,NaN,28780
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,NaN,28800
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,NaN,28820
5,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,NaN,28840
6,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,NaN,28860
7,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,0.459961,28880
8,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,1.559570,28900
9,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,train,2011-01-29,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,3.169922,28920


In [35]:
data.dropna(inplace=True)

In [36]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 7 to 59181089
Data columns (total 23 columns):
id              46881677 non-null object
item_id         46881677 non-null object
dept_id         46881677 non-null object
cat_id          46881677 non-null object
store_id        46881677 non-null object
state_id        46881677 non-null object
demand          46881677 non-null int16
part            46881677 non-null object
date            46881677 non-null object
wm_yr_wk        46881677 non-null int16
weekday         46881677 non-null object
wday            46881677 non-null int8
month           46881677 non-null int8
year            46881677 non-null int16
event_name_1    46881677 non-null object
event_type_1    46881677 non-null object
event_name_2    46881677 non-null object
event_type_2    46881677 non-null object
snap_CA         46881677 non-null int8
snap_TX         46881677 non-null int8
snap_WI         46881677 non-null int8
sell_price      46881677 non-null flo

In [38]:
def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
    
    encoder = preprocessing.LabelEncoder()
    data['id_encode'] = encoder.fit_transform(data['id'])
    
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1'\
           , 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        #print(feature)
        #print(data[feature].dtype)
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
        #print('success')
    data=reduce_mem_usage(data)
    return data

def simple_fe(data):

    
    data_fe = data[['id', 'demand']]
    
    window = 28
    periods = [7, 15, 30, 90]
    group = data_fe.groupby('id')['demand']
    
    # most recent lag data
    for period in periods:
        data_fe['demand_rolling_mean_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).mean())
        data_fe['demand_rolling_std_t' + str(period)] = group.transform(lambda x: x.shift(window).rolling(period).std())
        
    # reduce memory
    data_fe = reduce_mem_usage(data_fe)
    
    # get time features
    data['date'] = pd.to_datetime(data['date'])
    time_features = ['year', 'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear']
    dtype = np.int16
    for time_feature in time_features:
        data[time_feature] = getattr(data['date'].dt, time_feature).astype(dtype)
        
    # concat lag and rolling features with main table
    lag_rolling_features = [col for col in data_fe.columns if col not in ['id', 'demand']]
    data = pd.concat([data, data_fe[lag_rolling_features]], axis = 1)
    
    del data_fe
    gc.collect()
    
    return data

# define custom loss function
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
    hess = np.where(residual < 0, 2, 2 * 1.15)
    return grad, hess

# define custom evaluation metric
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * 1.15) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [14]:
def run_lgb(data, features, cat_features):
    
    # reset_index
    data.reset_index(inplace = True, drop = True)
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-04-24']
    y_train = x_train['demand']
    test = data[data['date'] >= '2016-04-25']

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.1,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        'min_data_per_leaf': 25,
        'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5}
    
    oof = np.zeros(len(x_train))
    preds = np.zeros(len(test))
    
       
    # GroupKFold by week, month to avoid leakage and overfitting (not entirely sure xD)
    kf = GroupKFold(5)
    # get subgroups for each week, year pair
    group = x_train['week'].astype(str) + '_' + x_train['year'].astype(str)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train, group)):
        print(f'Training fold {fold + 1}')
        train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], 
                                categorical_feature = cat_features)
        val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], 
                              categorical_feature = cat_features)
        
        # train with our custom loss function and evaluation metric
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                          valid_sets = [train_set, val_set], verbose_eval = 50, fobj = custom_asymmetric_train, 
                          feval = custom_asymmetric_valid)
    
        # predict oof
        oof[val_idx] = model.predict(x_train.iloc[val_idx][features])

        # predict test
        preds += model.predict(test[features]) / 5
        
        print('-'*50)
        print('\n')
        
    oof_rmse = np.sqrt(metrics.mean_squared_error(y_train, oof))
    print(f'Our out of folds rmse is {oof_rmse}')
        
    test = test[['id', 'date', 'demand']]
    test['demand'] = preds
    return test

In [ ]:
# function to get the predictions in the correct format
def predict(test, submission):
    predictions = pd.pivot(test, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission_custom_loss.csv', index = False)
    
# this is the main function that will run our entire program
def train_and_evaluate(data):
    
    # read data
    #print('Reading our data...')
    #data, submission = read_data()
    data = transform(data)
    data=reduce_mem_usage(data)
    gc.collect()
    data['date'] = pd.to_datetime(data['date'])
    # get amount of unique days in our data
    days = abs((data['date'].min() - data['date'].max()).days)
    # how many training data do we need to train with at least 2 years and consider lags
    need = 365 + 365 + 90 + 28
    print(f'We have {(days - 28)} days of training history')
    print(f'we have {(days - 28 - need)} days left')
    if (days - 28 - need) > 0:
        print('We have enought training data, lets continue')
    else:
        print('Get more training data, training can fail')
    
    # simple feature engineer
    print('Running simple feature engineering...')
    data = simple_fe(data)
    print('Removing first 118 days')
    # eliminate the first 118 days of our train data because of lags
    min_date = data['date'].min() + timedelta(days = 118)
    data = data[data['date'] > min_date]
    
    # define our numeric features and categorical features
    features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'year', 
                'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear', 'demand_rolling_mean_t7', 'demand_rolling_mean_t15', 'demand_rolling_mean_t30', 'demand_rolling_mean_t90',
                'demand_rolling_std_t7', 'demand_rolling_std_t15', 'demand_rolling_std_t30', 'demand_rolling_std_t90']
    
    cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
                    'event_name_2', 'event_type_2']
    
    print('-'*50)
    print('\n')
    print(f'Training model with {len(features)} features...')
    # run lgbm model with 5 GroupKFold (subgroups by year, month)
    test = run_lgb(data, features, cat_features)
    print('Save predictions...')
    # predict
    predict(test, submission)
        
# run our program
train_and_evaluate(data)

Mem. usage decreased to 2906.14 Mb (51.1% reduction)
Mem. usage decreased to 2906.14 Mb (0.0% reduction)
We have 1912 days of training history
we have 1064 days left
We have enought training data, lets continue
Running simple feature engineering...
Mem. usage decreased to 1520.13 Mb (58.5% reduction)
Removing first 118 days
--------------------------------------------------


Training model with 28 features...
Training fold 1
Training until validation scores don't improve for 50 rounds.
[50]	training's custom_asymmetric_eval: 6.62019	valid_1's custom_asymmetric_eval: 7.56132
[100]	training's custom_asymmetric_eval: 6.14727	valid_1's custom_asymmetric_eval: 7.27623
[150]	training's custom_asymmetric_eval: 5.9007	valid_1's custom_asymmetric_eval: 7.16816
[200]	training's custom_asymmetric_eval: 5.72531	valid_1's custom_asymmetric_eval: 7.10589
[250]	training's custom_asymmetric_eval: 5.58266	valid_1's custom_asymmetric_eval: 7.06287
[300]	training's custom_asymmetric_eval: 5.47424	valid_

[1050]	training's custom_asymmetric_eval: 4.59409	valid_1's custom_asymmetric_eval: 6.63014
[1100]	training's custom_asymmetric_eval: 4.55747	valid_1's custom_asymmetric_eval: 6.62576
[1150]	training's custom_asymmetric_eval: 4.52207	valid_1's custom_asymmetric_eval: 6.62021
[1200]	training's custom_asymmetric_eval: 4.48634	valid_1's custom_asymmetric_eval: 6.61457
[1250]	training's custom_asymmetric_eval: 4.45204	valid_1's custom_asymmetric_eval: 6.60823
[1300]	training's custom_asymmetric_eval: 4.41925	valid_1's custom_asymmetric_eval: 6.60561
[1350]	training's custom_asymmetric_eval: 4.38867	valid_1's custom_asymmetric_eval: 6.60117
[1400]	training's custom_asymmetric_eval: 4.35981	valid_1's custom_asymmetric_eval: 6.59851
[1450]	training's custom_asymmetric_eval: 4.33118	valid_1's custom_asymmetric_eval: 6.59572
[1500]	training's custom_asymmetric_eval: 4.30254	valid_1's custom_asymmetric_eval: 6.59136
[1550]	training's custom_asymmetric_eval: 4.27481	valid_1's custom_asymmetric_ev

In [ ]:
import seaborn as sns
feature_importances['average'] = feature_importances[[f'fold_{fold_n + 1}' for fold_n in range(folds.n_splits)]].mean(axis=1)
feature_importances.to_csv('feature_importances.csv')

plt.figure(figsize=(16, 12))
sns.barplot(data=feature_importances.sort_values(by='average', ascending=False).head(20), x='average', y='feature');
plt.title('20 TOP feature importance over {} folds average'.format(folds.n_splits));